In [2]:
import numpy as np
from math import pi, sqrt, tanh
# import matplotlib.pyplot as plt
# from scipy.linalg import block_diag
# from ipywidgets import interact
# from os import path

# import plotly.graph_objects as go
# import pandas as pd

from quimb import *
import time


In [2]:
def p_transpose_2(V,Na,Nb):
# partial transpose with respect to subsystem 2
# the basis of NaxNb density matrix is defined by Nb*(i-1)+j,
# i,j=1,2,3 spans the Hilbert space of subsystem 1 and 2 respectively
    U=np.zeros((Na*Nb,Na*Nb), dtype=np.complex128)
    for i_1 in range(Na):
        for i_2 in range(Na):
#             print(V[Nb*i_1:Nb*(i_1+1),Nb*i_2:Nb*(i_2+1)].shape)
            U[Nb*i_1:Nb*(i_1+1),Nb*i_2:Nb*(i_2+1)]=np.transpose(V[Nb*i_1:Nb*(i_1+1),Nb*i_2:Nb*(i_2+1)])

    return U

def vNent_calc(rho,epsilon=1e-9):
    chi0 =np.linalg.eigvalsh(rho)
    chi1=chi0[np.nonzero(np.abs(chi0)>epsilon)]
    chi2=chi1[np.nonzero(np.abs(chi1-1)>epsilon)]
    return -np.sum(chi2*np.log(chi2))


In [3]:
from scipy.integrate import quad, nquad

def Gz(z,a,b):
    '''Direct solution to G(z), faster but has numerical instability for large V
    a=  Lb/L2
    b=  Lb*L1/L2
    '''
    a2= (b-1)/z +1j*0
    a1= (a-z)/z +1j*0
    a0= 1/z +1j*0
    Q= (3*a1-a2**2)/9 +1j*0
    R= (9*a2*a1-27*a0-2*a2**3)/54 +1j*0
    D= Q**3 + R**2 +1j*0
    S= (R+D**0.5 +1j*0)**(1/3)
    T= (R-D**0.5 +1j*0)**(1/3)
    z4= -a2/3 + (1-1j*sqrt(3))*Q/S/2-(1+1j*sqrt(3))/2*S
    return np.abs(np.imag(z4))

def integrand(z, a, b):
    return Gz(z,a,b)*np.abs(z)

def neg_calc(a,b):
    '''Calculates logneg by \int P(z)|z| dz
    where P(z) is spectral density'''
    options={'limit':500}
    integral= nquad(integrand, [[-np.inf, np.inf]], args=(a, b),opts=[options,options])[0]
    return integral


# log negativity

In [42]:
Nrep=1
r=2 # local Hilbert space dim

La = 3 
Lb = 3
Lc = 3
Ld = 1
Lt = La+Lb+Lc+Ld

i_a = range(La)
i_b = range(La,La+Lb)
i_c = range(La+Lb,La+Lb+Lc)
i_d = range(La+Lb+Lc,Lt)

ln_ab=np.zeros(Nrep)
ln_ac=np.zeros(Nrep)
ln_bc=np.zeros(Nrep)
ln_ad=np.zeros(Nrep)
ln_bd=np.zeros(Nrep)
ln_cd=np.zeros(Nrep)
ln_ab_d=np.zeros(Nrep)
ln_ac_d=np.zeros(Nrep)
ln_bc_d=np.zeros(Nrep)
ln_ab_c=np.zeros(Nrep)

mi_ab=np.zeros(Nrep)
mi_ac=np.zeros(Nrep)
mi_bc=np.zeros(Nrep)
mi_ad=np.zeros(Nrep)
mi_bd=np.zeros(Nrep)
mi_cd=np.zeros(Nrep)
mi_ab_d=np.zeros(Nrep)
mi_ac_d=np.zeros(Nrep)
mi_bc_d=np.zeros(Nrep)
mi_ab_c=np.zeros(Nrep)
ent_a=np.zeros(Nrep)
ent_b=np.zeros(Nrep)
ent_c=np.zeros(Nrep)
ent_d=np.zeros(Nrep)

for i_r in range(Nrep):
    print(i_r,end="\r")
    psi = rand_ket(r**Lt)
#     psi = np.random.randn(r**Lt)+1j*np.random.randn(r**Lt)
#     psi = normalize(psi)
    
    ln_ab[i_r]= logneg_subsys(psi,dims=[r]*(Lt),sysa=i_a,sysb=i_b)
    ln_ac[i_r]= logneg_subsys(psi,dims=[r]*(Lt),sysa=i_a,sysb=i_c)
    ln_bc[i_r]= logneg_subsys(psi,dims=[r]*(Lt),sysa=i_b,sysb=i_c)
    ln_ad[i_r]= logneg_subsys(psi,dims=[r]*(Lt),sysa=i_d,sysb=i_a)
    ln_bd[i_r]= logneg_subsys(psi,dims=[r]*(Lt),sysa=i_d,sysb=i_b)
    ln_cd[i_r]= logneg_subsys(psi,dims=[r]*(Lt),sysa=i_d,sysb=i_c)
    ln_ab_c[i_r]= logneg_subsys(psi,dims=[r]*(Lt),sysa=i_c,sysb=np.concatenate((i_a,i_b)))
    ln_ab_d[i_r]= logneg_subsys(psi,dims=[r]*(Lt),sysa=i_d,sysb=np.concatenate((i_a,i_b)))
    ln_bc_d[i_r]= logneg_subsys(psi,dims=[r]*(Lt),sysa=i_d,sysb=np.concatenate((i_b,i_c)))
    ln_ac_d[i_r]= logneg_subsys(psi,dims=[r]*(Lt),sysa=i_d,sysb=np.concatenate((i_a,i_c)))

    
    mi_ab[i_r]= mutinf_subsys(psi,dims=[r]*(Lt),sysa=i_a,sysb=i_b)
    mi_ac[i_r]= mutinf_subsys(psi,dims=[r]*(Lt),sysa=i_a,sysb=i_c)
    mi_bc[i_r]= mutinf_subsys(psi,dims=[r]*(Lt),sysa=i_b,sysb=i_c)
    mi_ad[i_r]= mutinf_subsys(psi,dims=[r]*(Lt),sysa=i_d,sysb=i_a)
    mi_bd[i_r]= mutinf_subsys(psi,dims=[r]*(Lt),sysa=i_d,sysb=i_b)
    mi_cd[i_r]= mutinf_subsys(psi,dims=[r]*(Lt),sysa=i_d,sysb=i_c)
    mi_ab_c[i_r]= mutinf_subsys(psi,dims=[r]*(Lt),sysa=i_c,sysb=np.concatenate((i_a,i_b)))
    mi_ab_d[i_r]= mutinf_subsys(psi,dims=[r]*(Lt),sysa=i_d,sysb=np.concatenate((i_a,i_b)))
    mi_bc_d[i_r]= mutinf_subsys(psi,dims=[r]*(Lt),sysa=i_d,sysb=np.concatenate((i_b,i_c)))
    mi_ac_d[i_r]= mutinf_subsys(psi,dims=[r]*(Lt),sysa=i_d,sysb=np.concatenate((i_a,i_c)))

    ent_a[i_r] = entropy_subsys(psi,dims=[r]*(Lt),sysa=i_a)
    ent_b[i_r] = entropy_subsys(psi,dims=[r]*(Lt),sysa=i_b)
    ent_c[i_r] = entropy_subsys(psi,dims=[r]*(Lt),sysa=i_c)
    ent_d[i_r] = entropy_subsys(psi,dims=[r]*(Lt),sysa=i_d)

print("A-B: %.1f, %.1f" % (np.mean(ln_ab),np.mean(mi_ab)))
print("B-C: %.1f, %.1f" % (np.mean(ln_bc),np.mean(mi_bc)))
print("A-C: %.1f, %.1f" % (np.mean(ln_ac),np.mean(mi_ac)))
print("AB-C: %.1f" % (np.mean(ln_ab_c)))
print("LN D-one: %.1f, %.1f, %.1f" % (np.mean(ln_ad),np.mean(ln_bd),np.mean(ln_cd)))
print("MI D-one: %.1f, %.1f, %.1f" % (np.mean(mi_ad),np.mean(mi_bd),np.mean(mi_cd)))
print("LN D-two: %.1f, %.1f, %.1f" % (np.mean(ln_ab_d),np.mean(ln_bc_d),np.mean(ln_ac_d)))
print("ent A: %.1f" % (np.mean(ent_a)))
print("ent B: %.1f" % (np.mean(ent_b)))
print("ent C: %.1f" % (np.mean(ent_c)))
print("ent D: %.1f" % (np.mean(ent_d)))
# print("AB:",np.mean(ln_ab),np.mean(mi_ab),np.mean(mi_ab)/2-np.mean(ln_ab))
# print("AC:",np.mean(ln_ac),np.mean(mi_ac))
# print("AB-C:",np.mean(ln_ab_c))
# # print("BC:",np.mean(ln_bc),np.mean(mi_bc))
# print("D-one:",np.mean(ln_ad),np.mean(ln_bd),np.mean(ln_cd))
# print("MI D-one:",np.mean(mi_ad),np.mean(mi_bd),np.mean(mi_cd))
# print("D-two:",np.mean(ln_ab_d),np.mean(ln_bc_d),np.mean(ln_ac_d))
# print("D:", np.mean(ent_d))

A-B: 0.9, 2.1
B-C: 0.9, 2.1
A-C: 0.9, 2.1
AB-C: 2.9
LN D-one: 0.0, 0.0, 0.0
MI D-one: 0.1, 0.1, 0.1
LN D-two: 0.9, 0.9, 0.9
ent A: 3.0
ent B: 3.0
ent C: 3.0
ent D: 1.0


In [106]:
print("AB:",np.mean(ln_ab),np.mean(mi_ab),np.mean(mi_ab)/2-np.mean(ln_ab))
print("AC:",np.mean(ln_ac),np.mean(mi_ac))
print("AB-C:",np.mean(ln_ab_c))
# print("BC:",np.mean(ln_bc),np.mean(mi_bc))
print("D-one:",np.mean(ln_ad),np.mean(ln_bd),np.mean(ln_cd))
print("D-two:",np.mean(ln_ab_d),np.mean(ln_bc_d),np.mean(ln_ac_d))
print("D:", np.mean(ent_d))

AB: 0.0 0.0 0.0
AC: 0.8615013060803606 0.0
AB-C: 2.917438225293007
D-one: 0.0 0.0 0.0
D-two: 0.9407710462554771 0.9207185181054601 0.9460671073144041
D: 0.998898786569263


In [22]:
L = 4
r = 2
psi = np.zeros(r**L)
psi[0]=1
psi[3]=1
psi[12]=1
psi[-1]=1
psi = normalize(psi)
ln_a= logneg_subsys(psi,dims=[r]*L,sysa=[1],sysb=[2,3])
mi_a= mutinf_subsys(psi,dims=[r]*L,sysa=[1],sysb=[2,3])
print(ln_a)
print(mi_a)

0.0
0.0


In [19]:
r=2
L = 4
psi = np.zeros(r**L)
psi[-1]= sqrt(2)
psi[1]=1
psi[2]=1
psi[4]=1
psi[8]=1
psi = normalize(psi)
ln_a= logneg_subsys(psi,dims=[r]*L,sysa=[0],sysb=[2])
mi_a= mutinf_subsys(psi,dims=[r]*L,sysa=[0],sysb=[1])
print(ln_a)
print(mi_a)

0.0
0.4150374992788437


# reduced density matrix

In [29]:
Nrep=1
r=2 # local Hilbert space dim

La = 2 
Lb = 2
Lc = 2
Ld = 1
Lt = La+Lb+Lc+Ld

i_a = range(La)
i_b = range(La,La+Lb)
i_c = range(La+Lb,La+Lb+Lc)
i_d = range(La+Lb+Lc,Lt)

psi = rand_ket(r**Lt)
rho = partial_trace(psi,dims=[2]*Lt,keep=np.arange(0,La+Ld))
np.set_printoptions(precision=2, suppress=True, threshold=5)
print(np.array(np.abs(rho)))

[[0.09 0.03 0.03 ... 0.02 0.   0.02]
 [0.03 0.09 0.03 ... 0.01 0.04 0.03]
 [0.03 0.03 0.15 ... 0.04 0.04 0.02]
 ...
 [0.02 0.01 0.04 ... 0.15 0.04 0.06]
 [0.   0.04 0.04 ... 0.04 0.17 0.07]
 [0.02 0.03 0.02 ... 0.06 0.07 0.13]]


In [28]:
np.shape(rho)

(8, 8)

### 